In [5]:
import configparser
import psycopg2
import requests as r 
import json 
from os import write
import requests
from requests.api import request
import pandas as pd
import csv, json
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from dotenv import load_dotenv
import boto3
import os
load_dotenv()

True

In [25]:
def create_SparkSession():
    """[summary]

    Returns:
        [type]: [description]
    """
    spark = SparkSession.builder.\
    config("spark.jars.repositories", "https://repos.spark-packages.org/").\
    config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()
    return spark

spark = create_SparkSession()


In [26]:
spark

In [4]:


def create_SparkSession():
    """[summary]

    Returns:
        [type]: [description]
    """
    spark = SparkSession.builder.\
    config("spark.jars.repositories", "https://repos.spark-packages.org/").\
    config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()
    return spark


def binance_BtcUSDT_minute(data):
    """[
        - Take data CSV format and manipulate the spark dataframe
        - Capitalize first letter of each column
        - Round float to ceil and single digit
        ]

    Args:
        data ([type]): [description]

    Returns:
        [type]: [description]
    """
    spark = create_SparkSession()
    payload = spark.read.option('header', True).csv(data)
    payload = payload.select([F.col(col).alias(col.replace(' ','_')) for col in payload.columns])
    payload = payload.toDF(*[i.capitalize() for i in payload.columns])
    
    collect_columns=['Open', 'High', 'Low','Close', 'Volume_btc', 'Tradecount', 'Volume_usdt']
    
    for col1 in payload.columns:
        for col2 in collect_columns:
            if col1==col2:
                payload = payload.withColumn(col1, round(col1, 1))
    return payload


def create_boto3_session():
    session = boto3.Session\
    (
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key= os.getenv('AWS_SECRET_ACCESS_KEY')
    )
    return session


def upload_to_s3(bucketname, local_file_path, s3_file_path):
    session = create_boto3_session()
    s3 = boto3.resource('s3')
    try:
        s3.meta.client.upload_file(local_file_path, bucketname, s3_file_path)
        print('Success!')
    except Exception as e:
        print(f'{e}\n Fail!')
        

def download_from_s3(s3_bucket, s3_file_path, local_file_path):
    session = create_boto3_session()
    try:
        s3 = boto3.resource('s3')
        my_bucket = s3.Bucket(s3_bucket)
        my_bucket.download_file(s3_file_path, local_file_path)
        print('Success!')
    except Exception as e:
        print(f'{e}\n Fail!')

In [3]:
bucket_name = 'tien-duong1151'
local_file = 'data/exchange_info.csv'
s3_file_path = 'data/new_exchange_info.csv'
upload_to_s3(bucketname=bucket_name, local_file_path=local_file, s3_file_path=s3_file_path)

Success!


In [5]:
local_file = 'data/new_exchange_info_2.csv'
download_from_s3(bucket_name, s3_file_path, local_file )

Success!


In [2]:
def download_from_s3(s3_bucket, s3_file_path, local_file_path):
    
    try:
        s3 = boto3.resource('s3')
        my_bucket = s3.Bucket(s3_bucket)
        my_bucket.download_file(s3_file_path, local_file_path)
        print('Success!')
    except Exception as e:
        print(f'{e}\n Fail!')

aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key= os.getenv('AWS_SECRET_ACCESS_KEY')
s3_bucket_name = os.getenv('S3_BUCKET_NAME')
s3_file_path = 'data/coins_data.csv'

download_from_s3(s3_bucket=s3_bucket_name, s3_file_path=s3_file_path, local_file_path='data/test_file.csv')

    # binance_btc_file = 'data/binance_btc.csv'
    # bucket_name=s3_bucket_name
    # create_boto3_session(aws_key=aws_access_key_id, aws_secret_key=aws_secret_access_key)
    # upload_to_s3(bucketname=bucket_name, local_file_path=binance_btc_file, s3_file_path=binance_btc_file)
    # print("Successfully Uploaded to S3!")
    




Success!
